
### ***Weighted Sum Method Bicriteria Algorithm***
***Based on Aneja, Y.P. & Nair, P.K. (1979)***  
**coded by: Erik Maldonado Ascanio**

*1) Install gurobi and import packages*



In [ ]:
!pip install gurobipy
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt

*2) Input values and ranges*

In [ ]:
try:
  option = int(input('Select 1 for random test values, 2 for predefined test values or 3 to input data:'))

  if option == 1:

    size = int(input('Enter the number of variables:'))

    input_r1 = input('Range 1: enter min and max values in objective 1 (separated by comma):')
    minr1, maxr1 = input_r1.split(",")
    r1 = np.random.randint(int(minr1), int(maxr1), size=size)

    input_r2 = input('Range 2: enter min and max values in objective 2 (separated by comma):')
    minr2, maxr2 = input_r2.split(",")
    r2 = np.random.randint(int(minr2), int(maxr2), size=size)

    input_r3 = input('Range 3: enter min and max values in constraint (separated by comma):')
    minr3, maxr3 = input_r3.split(",")
    r3 = np.random.randint(int(minr3), int(maxr3), size=size)

  elif option == 2:
    size = 15
    r1 = [19, 5, 1, 14, 4, 1, 11, 7, 3, 4, 12, 6, 15, 9, 10]
    r2 = [4, 6, 3, 9, 4, 8, 19, 6, 6, 5, 10, 8, 9, 15, 7]
    r3 = [13, 9, 14, 8, 4, 10, 8, 9, 5, 6, 6, 3, 9, 8, 10]

  else:
    while True:
      input_r1 = input("Range 1: enter objective 1 values (separated by comma): ")
      r1 = list(map(int, input_r1.split(",")))
      input_r2 = input("Range 2: enter objective 2 values (separated by comma): ")
      r2 = list(map(int, input_r2.split(",")))
      input_r3 = input("Range 3: enter constraint values (separated by comma): ")
      r3 = list(map(int, input_r3.split(",")))
      if len(r1) == len(r2) and len(r2) == len(r3):
        size = len(r1)
        break
      else:
        print('Ranges are not the same lenght')

  while True:
    capacity = int(input("Enter the capacity: "))
    r3min = min(r3)
    r3sum = sum(r3)
    if capacity > r3min and capacity < r3sum:
      break
    else:
      print('Capacity value not valid.')

  print('Range 1 (objective 1): ', r1)
  print('Range 2 (objective 1): ', r2)
  print('Range 3 (constraint): ', r3)
except:
  print('Input error, please check the values.')


*3) Optimization Model*

In [ ]:
pointslist = list()

model = gp.Model()
x = model.addVars(size, name="x", vtype=gp.GRB.BINARY)

z1 = gp.quicksum(r1[i] * x[i] for i in range(size))
z2 = gp.quicksum(r2[i] * x[i] for i in range(size))
constraint = gp.quicksum(r3[i] * x[i] for i in range(size))
model.addConstr(constraint <= capacity)
model.setParam('OutputFlag',False)

# STEP 0
model.setObjective(z1, gp.GRB.MAXIMIZE)
model.optimize()
z11 = model.objval

tmpconstr = model.addConstr(z1 >= z11)
model.setObjective(z2, gp.GRB.MAXIMIZE)
model.optimize()
z21 = model.objval
model.remove(tmpconstr)

pointslist.append((z11,z21))
k = 1

model.setObjective(z2, gp.GRB.MAXIMIZE)
model.optimize()
z22 = model.objval

tmpconstr = model.addConstr(z2 >= z22)
model.setObjective(z1, gp.GRB.MAXIMIZE)
model.optimize()
z12 = model.objval
model.remove(tmpconstr)

pointslist.append((z12,z22))

if pointslist[0] == pointslist[1]:
  print('The solution for z1, z2 is :', pointslist[1],'And the x values are:')
  for v in model.getVars():
    print(v.varName, ': ',v.x)
else:
  k += 1
  Lindex = [(1,2)]

  # STEP 1

  #choose a random L tuple
  while True:
      if len(Lindex) > 1:
        Lrnd = np.random.randint(0, len(Lindex)-1)
      else:
        Lrnd=0

      r = Lindex[Lrnd][0]
      s = Lindex[Lrnd][1]
      a1rs = abs(pointslist[s-1][1] - pointslist[r-1][1])
      a2rs = abs(pointslist[s-1][0] - pointslist[r-1][0])

      zbar = a1rs * z1 + a2rs * z2

      model.setObjective(zbar, gp.GRB.MAXIMIZE)
      model.optimize()
      zbar1 = z1.getValue()
      zbar2 = z2.getValue()

      if (zbar1,zbar2) == (pointslist[r-1][0],pointslist[r-1][1]) or (zbar1,zbar2) == (pointslist[s-1][0],pointslist[s-1][1]):
        Lindex.remove((r,s))
      else:
        pointslist.append((zbar1,zbar2))
        k+=1
        Lindex.remove((r,s))
        Lindex.append((r,k))
        Lindex.append((k,s))

      if len(Lindex)==0:
        break

  print("Solution points: ",pointslist)

*4) Dot Diagram*

In [ ]:
x_coords = [x for x, y in pointslist]
y_coords = [y for x, y in pointslist]
plt.plot(x_coords, y_coords, '.')
plt.show()